In [1]:
#Analyzing Artist Data from Nobody Gets It Records

#Important Imports
import spotipy as sp
import sys
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import numpy as np
import random
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import time
import sys
import urllib
from sklearn.preprocessing import MultiLabelBinarizer
import requests
from urllib.parse import urlencode
import base64
import webbrowser

In [2]:
#Connecting to the Spotify Web API, should ask user to sign in if they haven't done so already 
client_id = "80e5d278fa0d44a19e7a76b745bb8d2f"
client_secret = "35912046a82545a19e49c17a4f92ef5e"

auth_headers = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "https://localhost:1111/callback",
    "scope": "user-library-read"
}

webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))
#If using local host, take code from the search bar and input it into the code string on the next cell 


True

In [3]:
code = 'AQDJNmP2FlkeSZ-gySSwDMirDBEFI9oo44oxdxE0lf6pvGUetb79b-IDgtBqdDg-LbvIWuqeJouDXW-YCNg_Pz8R8IyjNxwcgedc7EkNk8E61WI5i_ZXkYJhcXazD66S5fVuYXltx7Eqg9uZJUAqxQQCZQxTf3wya7WZn9em7jJl-jSkqzrSlfNfv-5-XCxBfBhtHHAS'

In [4]:
#Authorizing the code and returning a token 
encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "https://localhost:1111/callback"
}

r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)

In [5]:
token = r.json()["access_token"]

print(token)

BQCtzBqxkF-NCToZsUVOgovuJlMwjrv-A5zKG9VYIMmG8vjlEcuTbVCjapcD1MSBzkJJpMs3nyrnTNGGMVKy9fX9E7Fw3gLBxikVB9PSuir9Z-9DfbpX38Vos6F9tNIuNRz75vsytErpVJMapBX0t6Nw_dI71S4pygKIBORl_fRiYhhdp6FDTG-aSXKj4boUwjHzzVs-4hQZNjsEMI8


In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
s = sp.Spotify(client_credentials_manager=client_credentials_manager)


In [31]:
artist_name = []
track_name = []
track_popularity = []
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = s.search(q='genre:Indie Rock', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])

In [32]:
#Create a DataFrame of 1000 Random Indie Rock Songs
indie_df = pd.DataFrame({'artist_name': artist_name, 'track_name' : track_name, 'artist_id': artist_id, 'track_id' : track_id, 'track_popularity' : track_popularity})
indie_df

,artist_name,track_name,artist_id,track_id,track_popularity
0,TV Girl,Lovers Rock,0Y6dVaC9DZtPNH4591M42W,6dBUzqjtbnIa1TwYbyw5CM,89
1,Tyler Childers,Rock Salt and Nails (Live),13ZEDW6vyBF12HYcZRr4EV,20dkebBtPR4AJ8vVos41Gw,69
2,Mura Masa,Love$ick (feat. A$AP Rocky),5Q81rlcTFh3k6DQJXPdsot,3sTN90bIP2cJ1783ctHykO,76
3,G-Eazy,No Limit,02kJSzxNuaWGqwubyUba0Z,2DQ1ITjI0YoLFzuADN1ZBW,72
4,SALES,Pope Is a Rockstar,6vVztIuqdDHvYWxOEXCzjN,0So2sgVa8aJiARPl2P29u2,72
...,...,...,...,...,...
995,Broken Social Scene,World Sick,7lOJ7WXyopaxri0dbOiZkd,6Ko9Gi0CD77okK0hlW9SoL,15
996,Tea Leaf Green,Faced With Love,4n3xlLBs7G1x9tJDXFD4G4,73gDzig6Dq1FSfr1aRKTTt,15
997,The Evaporators,I Feel Like A Fat Frustrated Fuck,3E6zfYGiiMDHNEMUPnKvQs,0mph4yYswhlUsprIfTqCXo,17
998,The Dead Milkmen,Rocket Ship,2fhngmJMaaKcdhXHCa4cPK,6LmKS1r4f4c03xVCtuK2CF,12


In [34]:
artist_popularity = []
artist_genres = []
artist_followers = []
for a_id in indie_df.artist_id:
  artist = s.artist(a_id)
  artist_popularity.append(artist['popularity'])
  artist_genres.append(artist['genres'])
  artist_followers.append(artist['followers']['total'])

In [36]:

indie_df = indie_df.assign(artist_popularity=artist_popularity, artist_genres=artist_genres, artist_followers=artist_followers)
indie_df

,artist_name,track_name,artist_id,track_id,track_popularity,artist_popularity,artist_genres,artist_followers
0,TV Girl,Lovers Rock,0Y6dVaC9DZtPNH4591M42W,6dBUzqjtbnIa1TwYbyw5CM,89,79,[pov: indie],2636620
1,Tyler Childers,Rock Salt and Nails (Live),13ZEDW6vyBF12HYcZRr4EV,20dkebBtPR4AJ8vVos41Gw,69,77,"[kentucky indie, kentucky roots, outlaw country]",1796615
2,Mura Masa,Love$ick (feat. A$AP Rocky),5Q81rlcTFh3k6DQJXPdsot,3sTN90bIP2cJ1783ctHykO,76,62,"[channel islands indie, hyperpop, indie soul, ...",646887
3,G-Eazy,No Limit,02kJSzxNuaWGqwubyUba0Z,2DQ1ITjI0YoLFzuADN1ZBW,72,75,"[indie pop rap, oakland hip hop, pop rap, rap]",5171477
4,SALES,Pope Is a Rockstar,6vVztIuqdDHvYWxOEXCzjN,0So2sgVa8aJiARPl2P29u2,72,59,"[indie pop, orlando indie, pov: indie]",530397
...,...,...,...,...,...,...,...,...
995,Broken Social Scene,World Sick,7lOJ7WXyopaxri0dbOiZkd,6Ko9Gi0CD77okK0hlW9SoL,15,48,"[alternative rock, baroque pop, canadian indie...",349596
996,Tea Leaf Green,Faced With Love,4n3xlLBs7G1x9tJDXFD4G4,73gDzig6Dq1FSfr1aRKTTt,15,32,"[bay area indie, jam band, progressive bluegrass]",40053
997,The Evaporators,I Feel Like A Fat Frustrated Fuck,3E6zfYGiiMDHNEMUPnKvQs,0mph4yYswhlUsprIfTqCXo,17,13,[vancouver indie],6090
998,The Dead Milkmen,Rocket Ship,2fhngmJMaaKcdhXHCa4cPK,6LmKS1r4f4c03xVCtuK2CF,12,44,"[alternative rock, cowpunk, hardcore punk, phi...",128614
